<a href="https://colab.research.google.com/github/ipassynk/dating-match-fine-tuning/blob/main/eval/baseline_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers
!pip install sklearn
!pip install umap
!pip install hdbscan
!pip install matplotlib
!pip install plotly
!pip install scipy

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Preparing metadata (setup.py) ... done
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3541 sha256=f172d0ac075dd78d667cea63091ded86714c0801959f6d190ef7f1e58509d304
  Stored in directory: /root/.cache/pip/wheels/48/4a/1c/1d511cbb0413a448d8546e958f8e82b98d9bb493038d19ece2
Successfully built umap


In [16]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json
import numpy as np
import pandas as pd
from scipy import stats

In [3]:
model_name ='all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
def read_file(file_name):
    positives = [];
    negatives = [];

    with open(file_name, 'r') as data:
        lines = data.readlines()
        for line in lines:
            item = json.loads(line);
            sentence1 = item['text_1'].split(': ')[1]
            sentence2 = item['text_2'].split(': ')[1]
            label = item['label']
            el = {'sentence1': sentence1, 'sentence2': sentence2} # Corrected key for sentence2
            if label == 1:
                positives.append(el)
            else:
                negatives.append(el)
    return  pd.DataFrame(positives), pd.DataFrame(negatives)

positives, negatives = read_file("dating_pairs.jsonl");
print(positives.iloc[0])

sentence1                 I love lazy Sundays
sentence2    I'm all about meeting new people
Name: 0, dtype: object


In [14]:
sentences1 = [row['sentence1'] for index, row in positives.iterrows()];
sentences2 = [row['sentence2'] for index, row in positives.iterrows()];
embeddings1_positives = model.encode(sentences1, convert_to_numpy=True, show_progress_bar=True)
embeddings2_positives = model.encode(sentences2, convert_to_numpy=True, show_progress_bar=True)
similarity_positives_matrix = cosine_similarity(embeddings1_positives, embeddings2_positives)
similarity_positives = np.diag(similarity_positives_matrix)

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [15]:

sentences1 = [row['sentence1'] for index, row in negatives.iterrows()];
sentences2 = [row['sentence2'] for index, row in negatives.iterrows()];
embeddings1_negatives = model.encode(sentences1, convert_to_numpy=True, show_progress_bar=True)
embeddings2_negatives = model.encode(sentences2, convert_to_numpy=True, show_progress_bar=True)
similarity_negatives_matrix = cosine_similarity(embeddings1_negatives, embeddings2_negatives)
similarity_negatives = np.diag(similarity_negatives_matrix)

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

t-test to check if the average similarity of sim_positive is significantly higher than that of sim_negative.

In [17]:
# Independent t-test
t_stat, p_value = stats.ttest_ind(similarity_positives, similarity_negatives)
print("t-statistic:", t_stat)
print("p-value:", p_value)

t-statistic: -3.657630873374173
p-value: 0.00026565557351647954


Even if the difference is statistically significant, you also want to know how big it is. That’s what Cohen’s d measures:

In [18]:
def cohens_d(a, b):
    mean_diff = np.mean(a) - np.mean(b)
    pooled_std = np.sqrt((np.std(a, ddof=1)**2 + np.std(b, ddof=1)**2) / 2)
    return mean_diff / pooled_std

d = cohens_d(similarity_positives, similarity_negatives)
print("Cohen's d:", d)

Cohen's d: -0.21276489


To calculate the Compatibility Margin, you use the difference between the average similarity of compatible pairs and incompatible pairs.

Interpretation

Higher margin → better separation between compatible and incompatible pairs (model performs well).

Smaller margin → overlap between groups (model struggles to distinguish them).

Negative margin → incompatible pairs are more similar than compatible ones — indicates an issue.

In [19]:
def calculate_compatibility_margin(sim_positives, sim_negatives):
  mean_compatible = np.mean(sim_positives)
  mean_incompatible = np.mean(sim_negatives)
  compatibility_margin = mean_compatible - mean_incompatible
  return compatibility_margin

compatibility_margin = calculate_compatibility_margin(similarity_positives, similarity_negatives)
print("Compatibility Margin:", compatibility_margin)


Compatibility Margin: -0.044882238
